In [ ]:
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
conf_threshold = 0.6
save_csv = True

# Json Daten Megedetectron lesen

In [ ]:
train_features_label = pd.read_csv("../eda/train_features_label.csv", index_col='id')

mega_json = json.load(open('train_features_output.json'))
mega_json.keys()

In [ ]:
dict_detection_cat = mega_json['detection_categories']
dict_detection_cat

In [ ]:
mega_detector_info = mega_json['info']
mega_detector_info

In [ ]:
df_images = pd.DataFrame(mega_json["images"])
df_images = df_images.reset_index(drop=True)
df_images

In [ ]:
# several bbox per image possible
df_images['detections'][0]

# Daten in geeignetes Format bringen

In [ ]:
df_images_detections = df_images['detections'].apply(pd.Series, dtype='object')
#display(df_images_detections.head(2))

df_images_clean = df_images.merge(df_images_detections, left_index=True, right_index=True).drop(columns='detections')
#display(df_images_clean.head(2))

df_images_clean = df_images_clean.melt(id_vars=['file', 'max_detection_conf']).sort_values('file')
#display(df_images_clean)

# remove nan values 
df_images_clean = df_images_clean.dropna(subset='value')
df_images_clean = df_images_clean.drop(columns='variable')

# expand category, conf, bbox
df_cat_conf_bbox = df_images_clean['value'].apply(pd.Series, dtype='object')
df_images_clean = df_images_clean.merge(df_cat_conf_bbox, left_index=True, right_index=True).drop(columns='value')

# take only detections for max detection conf
df_images_clean = df_images_clean[df_images_clean['max_detection_conf'] == df_images_clean['conf']]
df_images_clean = df_images_clean.reset_index(drop=True)

# leftjoin clean data
df_images_clean = pd.merge(df_images['file'], df_images_clean, on='file', how='left')

# cast datatypes
df_images_clean['category'] = df_images_clean['category'].astype('category')
df_images_clean['category'] = df_images_clean['category'].replace(dict_detection_cat)
df_images_clean

# change index to image name
df_images_clean['image_name'] = [image.replace('.jpg', '') for image in df_images_clean['file']]
df_images_clean.index = df_images_clean['image_name']
df_images_clean = df_images_clean.drop(columns='image_name')
df_images_clean

# boolean column for usefull detection
df_images_clean['conf_th'] = df_images_clean['max_detection_conf'] > conf_threshold
display(df_images_clean)

if save_csv: df_images_clean.to_csv('megadetector_image_detection_bbox.csv')

Unterschiedliche Shapes zwischen `df_images` und `df_images_clean` -> `16488` und `16491` ?

In [ ]:
df_images[~df_images['file'].isin(df_images_clean['file'])]


In [ ]:
df_images_clean[~df_images_clean['file'].isin(df_images['file'])]

# Untersuche Kategorien  
Das Modell des Megadetectors unterscheidet zwischen drei Kategorien: `Animal`, `Person`, `Vehicle`

In [ ]:
df_images_clean['category'].unique().to_list()

In [ ]:
df_images_clean['category'].value_counts().plot.bar()
cat_na = df_images_clean.shape[0]- df_images_clean['category'].value_counts().sum()
plt.title(f'Verteilung der detektierten Kategorien, na:{cat_na}')
plt.show()

In [ ]:
df_images_clean[df_images_clean['category'] == 'animal']['max_detection_conf'].plot.hist()
plt.title('Verteilung von max_detection_conf für Kategorie 1 (Animal)')
plt.xlabel('max_detection_conf')
plt.show()

In [ ]:
df_images_clean[df_images_clean['category'] == 'person']['max_detection_conf'].plot.hist()
plt.title('Verteilung von max_detection_conf für Kategorie 2 (Person)')
plt.xlabel('max_detection_conf')
plt.show()

In [ ]:
df_images_clean[df_images_clean['category'] == 'vehicle']['max_detection_conf'].plot.hist()
plt.title('Verteilung von max_detection_conf für Kategorie 3 (vehicle)')
plt.xlabel('max_detection_conf')
plt.show()

## Prüfe einzelne Bilder für die Klassen `person` und `vehicle`

In [ ]:
lst_img_cat1 = df_images_clean[df_images_clean['category'] == 'animal']['file']
lst_img_cat1 = lst_img_cat1.reset_index(drop=True)
lst_img_cat1 = [image_name.replace('.jpg', '') for image_name in lst_img_cat1]
lst_img_cat1

lst_img_cat2 = df_images_clean[df_images_clean['category'] == 'person']['file']
lst_img_cat2 = lst_img_cat2.reset_index(drop=True)
lst_img_cat2 = [image_name.replace('.jpg', '') for image_name in lst_img_cat2]
lst_img_cat2

lst_img_cat3 = df_images_clean[df_images_clean['category'] == 'vehicle']['file']
lst_img_cat3 = lst_img_cat3.reset_index(drop=True)
lst_img_cat3 = [image_name.replace('.jpg', '') for image_name in lst_img_cat3]
lst_img_cat3

path_img_train = '../competition_data/'
path_img_mega = './train_features_detection_th01/'

Random Ansicht `person`

In [ ]:
random_lst_img_cat2 = random.sample(lst_img_cat2, 6)

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

# iterate through each bad image
for idx, (img_id, ax) in enumerate(zip(random_lst_img_cat2, axes.flat)):
    # get image label
    img_label = train_features_label.loc[img_id]['label']
    # reads the filepath and returns a numpy array
    img = mpimg.imread(path_img_mega + str(img_id).upper() + '_detections.jpg')
    # get category
    cat, conf = df_images_clean.loc[img_id]['category'], df_images_clean.loc[img_id]['max_detection_conf'], 
    # plot etc
    ax.imshow(img)
    ax.set_title(f"{img_id} | {img_label} | {cat} | Conf: {conf:2f}", fontsize=8)

plt.tight_layout()
plt.show() 

Random Ansicht `vehicle`

In [ ]:
random_lst_img_cat3 = random.sample(lst_img_cat3, 6) 

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

# iterate through each bad image
for idx, (img_id, ax) in enumerate(zip(random_lst_img_cat3, axes.flat)):
    # get image label
    img_label = train_features_label.loc[img_id]['label']
    # reads the filepath and returns a numpy array
    img = mpimg.imread(path_img_mega + str(img_id).upper() + '_detections.jpg')
    # get category
    cat, conf = df_images_clean.loc[img_id]['category'], df_images_clean.loc[img_id]['max_detection_conf'], 
    # plot etc
    ax.imshow(img)
    ax.set_title(f"{img_id} | {img_label} | {cat} | Conf: {conf:2f}", fontsize=8)

plt.tight_layout()
plt.show()


## Random Ansicht `animal`

In [ ]:
random_lst_img_cat1 = random.sample(lst_img_cat1, 12)

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 15))

# iterate through each bad image
for idx, (img_id, ax) in enumerate(zip(random_lst_img_cat1, axes.flat)):
    # get image label
    img_label = train_features_label.loc[img_id]['label']
    # reads the filepath and returns a numpy array
    img = mpimg.imread(path_img_mega + str(img_id).upper() + '_detections.jpg')
    # get category
    cat, conf = df_images_clean.loc[img_id]['category'], df_images_clean.loc[img_id]['max_detection_conf'], 
    # plot etc
    ax.imshow(img)
    ax.set_title(f"{img_id} | {img_label} | {cat} | Conf: {conf:2f}", fontsize=8)

fig.tight_layout()
plt.show()

## Beispiele für Falschklassifikation mit hoher Confidence

In [ ]:
missclassification_findings = ['ZJ010981', 'ZJ014854', 'ZJ003542']
random_miss_class= random.sample(missclassification_findings, 3)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 12))

# iterate through each bad image
for idx, (img_id, ax) in enumerate(zip(random_miss_class, axes.flat)):
    # get image label
    img_label = train_features_label.loc[img_id]['label']
    # reads the filepath and returns a numpy array
    img = mpimg.imread(path_img_mega + str(img_id).upper() + '_detections.jpg')
    # get category
    cat, conf = df_images_clean.loc[img_id]['category'], df_images_clean.loc[img_id]['max_detection_conf'], 
    # plot etc
    ax.imshow(img)
    ax.set_title(f"{img_id} | {img_label} | {cat} | Conf: {conf:2f}", fontsize=8)

plt.tight_layout()
plt.show()